<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Dec 18 00:18:05 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install tensorflow-gpu --upgrade

# bert uncased base
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbu
!mkdir bert_base_uncased
!tar -xzf bbu -C bert_base_uncased

# bert cased base
!wget https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbc
!mkdir bert_base_cased
!tar -xzf bbc -C bert_base_cased

# albert base
!wget https://tfhub.dev/google/albert_base/1?tf-hub-format=compressed -O ab
!mkdir albert_base
!tar -xzf ab -C albert_base

# bert uncased large
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1?tf-hub-format=compressed -O blu
!mkdir bert_large_uncased
!tar -xzf blu -C bert_large_uncased

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.insert(1, '/content/drive/My Drive/GoogleQA')
import numpy as np
import pandas as pd
import os, gc
import codecs
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.backend import set_session
# from keras.callbacks import Callback
# from keras.models import Model, load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold, RepeatedKFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
# from albert_tokenization import FullTokenizer
from tokenizer import Tokenizer
from warmup_v2 import AdamWarmup, calc_train_steps
tf.get_logger().setLevel('ERROR') 
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
# from hyperopt import fmin, hp, tpe, STATUS_OK, Trials


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')
aux = pd.read_csv(data_path+'df_qa_extracted.csv')[['qa_id', 'question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes']]

target_col = train.columns.tolist()[11:42]
train['question'] = train['question_title'] + ' ' + train['question_body']
test['question'] = test['question_title'] + ' ' + test['question_body']
train = train.merge(aux, how='left', on='qa_id')
train = train.fillna(aux.median())
# def aux_build(df):
#     aux = pd.get_dummies(df['category'], drop_first=True)
#     aux['stackexchange'] = df.host.str.contains('stackexchange').astype('int32')
#     aux['stackoverflow'] = df.host.str.contains('stackoverflow').astype('int32')
#     return aux
# train_aux, test_aux = aux_build(train), aux_build(test)


In [0]:
MAXLEN = 512 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 4 #@param {type:'slider', min:1, max:32, step:1}
NUM_EPOCHS = 3
NUM_CLASSES = 30
LR = 5e-5
MIN_LR = 0
model_path = 'bert_base_uncased' #@param ['bert_base_uncased', 'bert_base_cased', 'bert_large_uncased', 'albert_base']
save_model = 'bbu' #@param ['bbu', 'bbc', 'blu', 'tmp', 'use']
CASED = False #@param ['True', 'False'] {type:"raw"}
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
# model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']
# target_q_col = train.columns.tolist()[11:32]
# target_a_col = train.columns.tolist()[32:42]


## Bert tokenizer

In [0]:
token_dict = {}
with codecs.open(os.path.join(model_path, 'assets/vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict, cased=CASED)

def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    ids, segments = [], []
    # q_title, q_body, answer = [], [], []
    for i in tqdm(range(len(data_df))):
        a, b = tokenizer.encode(data_df.loc[i, 'question'], data_df.loc[i, 'answer'])
        ids.append(a)
        segments.append(b)
    aux = data_df[['question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes']]
    aux = aux.apply(lambda x: (x - x.mean()) / x.std())             
    if branch == 'training':
        targets = data_df[target_col]
        # return [q_title, q_body], np.array(targets)
        return [ids, segments], np.array(aux, dtype='float32'), np.array(targets)
    else:
        return [ids, segments], np.array(aux, dtype='float32')
        # return [q_title, q_body]

In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    # L = [len(x) for x in X]
    # ML = min(max(L), MAXLEN)
    ML = MAXLEN
    out = np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)
    
def get_masks(X, padding=0):
    # L = [len(x) for x in X]
    # ML = min(max(L), MAXLEN)
    ML = MAXLEN
    out = np.array([np.concatenate([[1]*len(x), [padding] * (ML - len(x))]) if len(x) < ML else [1]*ML for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        # self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])
                # X3 = np.zeros_like(X1)

                aux = tf.convert_to_tensor([x[2] for x in d], dtype=tf.float32)
                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3]
                    # yield [X1, X2, X3, X4, X5, X6]
                else:
                    Y = tf.convert_to_tensor([x[3] for x in d], dtype=tf.float32)
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3], [Y, aux] 
                    # yield [X1, X2, X3, X4, X5, X6], Y


In [0]:
def model_build(len_train):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    global LR
    global MAXLEN
    global model_path
    # global train_aux
 
    q_in = keras.layers.Input(shape=(MAXLEN,), dtype=tf.int32, name="q_input_word_ids")
    q2_in = keras.layers.Input(shape=(MAXLEN,), dtype=tf.int32, name="q_input_masks")
    q3_in = keras.layers.Input(shape=(MAXLEN,), dtype=tf.int32, name="q_segment_ids")


    bert_layer = hub.KerasLayer(model_path, trainable=True)
    _, q_inputs  = bert_layer([q_in, q2_in, q3_in])
    # q_outputs = keras.layers.Lambda(lambda x: x[:, 0])(q_inputs)
    q_outputs = keras.layers.GlobalAveragePooling1D()(q_inputs)
    dense = keras.layers.Dropout(0.2)(q_outputs)

    # dense = keras.layers.concatenate([dense, aux_in])
    outputs = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dense)
    aux_out = keras.layers.Dense(4)(dense)
    model = keras.Model([q_in, q2_in, q3_in], [outputs, aux_out])

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
    )

    model.compile(
        loss=['binary_crossentropy', 'mean_squared_error'],
        loss_weights = [10, 0.25],
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=LR,
            min_lr=MIN_LR,
            ))
    #      optimizer=keras.optimizers.Adam(LR),
    # )

    return model

## train model

In [0]:
# pred = np.zeros((len(test), NUM_CLASSES))
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_body)]
# tr1, tr2 = train.loc[idx[0][0]], train.loc[idx[0][1]]
# tr1 = tr1.reset_index(drop=True)
# idx1 = [x for x in kf.split(tr1, groups=tr1.question_body)]


def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self):
        global NUM_EPOCHS
        global save_model
        super(keras.callbacks.Callback, self).__init__()
    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        score = compute_spearmanr(val_y, val_pred[0])
        print('Spearman - {:.5f}'.format(score))
        # if epoch>=NUM_EPOCHS-1:
        #     print('--Save Model--')
        #     self.model.save('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-{:}.h5'.format(save_model, save_model, i, epoch+1))


In [0]:
for i, (tr_idx, val_idx) in enumerate(idx[:1], 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    # tr_aux, val_aux = train_aux.loc[tr_idx], train_aux.loc[val_idx]
    tr_x, tr_aux, tr_y = convert_data(tr)
    val_x, val_aux, val_y = convert_data(val)
    
    model = model_build(len(tr))

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_aux, tr_y)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux, val_y)), branch='valid')
    # test_D = data_generator(list(zip(test_x[0], test_x[1])), branch='test')
    ieval = IntervalEval()
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )

    tf.keras.backend.get_session().close()
    cfg = tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    tf.reset_default_graph()
    gc.collect()
    tf.keras.backend.set_session(tf.Session(config=cfg))

Epoch 1/3
1061/1216 [=========================>....] - ETA: 1:58 - loss: 4.3655 - dense_2_loss: 0.4119 - dense_3_loss: 0.9876

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
q_input_word_ids (InputLayer)   [(None, 512)]        0                                            
__________________________________________________________________________________________________
q_input_masks (InputLayer)      [(None, 512)]        0                                            
__________________________________________________________________________________________________
q_segment_ids (InputLayer)      [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   q_input_word_ids[0][0]           
                                                                 q_input_masks[0][0]          

## Bert model stacking

In [0]:
# 0.8771 / 0.8779 / 0.8786 / 0.8774 / 0.8790
tr2_x, tr2_y = convert_data(tr2)
valid2_D = data_generator(list(zip(tr2_x[0], tr2_x[1], tr2_y)), branch='valid')
model = model_build(len(train), model='bert')
ensemble_inputs, ensemble_outputs = [], []
# update all layers in all models to not be trainable
for i in range(1, 11):
    # define filename for this ensemble
    filename = 'drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-3.h5'.format(save_model, save_model, i))

    # load model from file
    model.load_weights(filename)
    print('>loaded %s' % filename)

    model_pred = model.predict_generator(valid2_D.__iter__(), len(valid2_D), verbose=1)
    score = compute_spearmanr(tr2_y, model_pred)
    print('Spearman - {:.5f}'.format(score))
    for layer in model.layers:
        # make not trainable
        layer.trainable = False
        # rename to avoid 'unique layer name' issue
        layer.name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_inputs.append(model.input)
    # concatenate merge output from each model
    ensemble_outputs.append(model.output)
    # del model
    gc.collect()

# build model
merge = concatenate(ensemble_outputs)
hidden = Dense(100, activation='relu')(merge)
output = Dense(NUM_CLASSES, activation='softmax')(hidden)
stacked_model = Model(inputs=[x for y in ensemble_inputs for x in y], outputs=output)
# compile
decay_steps, warmup_steps = calc_train_steps(
    len(tr2),
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
)

stacked_model.compile(
    loss='binary_crossentropy',
    optimizer=AdamWarmup(
        decay_steps=decay_steps,
        warmup_steps=warmup_steps,
        lr=LR,
        min_lr=MIN_LR,
        ))



In [0]:
class data_generator_stacked:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])
                # X3 = np.zeros_like(X1)

                # aux = np.array([x[2] for x in d])
                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3] * 10
                    # yield [X1, X2, X3, X4, X5, X6]
                else:
                    Y = np.array([x[2] for x in d])
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3] * 10, Y
                    # yield [X1, X2, X3, X4, X5, X6], Y

# fit model
stack_D = data_generator_stacked(list(zip(tr2_x[0], tr2_x[1], tr2_y)))
stacked_model.fit_generator(
    stack_D.__iter__(),
    steps_per_epoch=len(stack_D),
    epochs=NUM_EPOCHS)

stack_valid_D = data_generator_stacked(list(zip(tr2_x[0], tr2_x[1], tr2_y)), branch='valid')
stacked_pred = stacked_model.predict_generator(stack_valid_D.__iter__(), len(stack_valid_D), verbose=1)
print('Spearman - {:.5f}'.format(compute_spearmanr(tr2_y, stacked_pred)))

del stacked_model
gc.collect()

## USE Model

In [0]:

# train['sentence'] = train['question'] + ' ' + train['answer']
embed1 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
# embed2 = hub.load('https://tfhub.dev/google/nnlm-en-dim128/2')

def UniversalEmbedding(x):
    return embed1(tf.squeeze(tf.cast(x, tf.string)))

# def NNLMEmbedding(x):
#     return embed2(tf.squeeze(tf.cast(x, tf.string)))

def use_model():
    
    q = keras.layers.Input(shape=(1,), dtype=tf.string)
    qb = keras.layers.Input(shape=(1,), dtype=tf.string)
    a = keras.layers.Input(shape=(1,), dtype=tf.string)

    def hidden_layer(input_layer):
        x = keras.layers.Lambda(UniversalEmbedding, output_shape=(512, ))(input_layer)
        # embed2 = keras.layers.Lambda(NNLMEmbedding, output_shape=(128, ))(input_layer)
        # embed = keras.layers.concatenate([embed1, embed2])
        x = keras.layers.Dropout(0.2)(x)
        x = keras.backend.expand_dims(x, axis=1)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # GP = keras.layers.GlobalMaxPooling1D()(x)
        # GP = keras.layers.Dropout(0.2)(GP)
        # AP = keras.layers.GlobalAveragePooling1D()(x)
        # AP = keras.layers.Dropout(0.2)(AP)
        # hidden = keras.layers.concatenate([GP, AP])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])

        x1 = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(256, return_sequences=True))(x)
        x2 = keras.layers.Bidirectional(keras.layers.CuDNNGRU(128, return_sequences=True))(x1)
        max_pool1 = keras.layers.GlobalMaxPooling1D()(x1)
        max_pool2 = keras.layers.GlobalMaxPooling1D()(x2)
        hidden = keras.layers.concatenate([max_pool1, max_pool2])
        return hidden

    q_embed, qb_embed, a_embed = hidden_layer(q), hidden_layer(qb), hidden_layer(a)
    all_embed = keras.layers.concatenate([q_embed, qb_embed, a_embed])
    dense = keras.layers.Dense(256, activation='relu')(all_embed)
    pred = keras.layers.Dense(30, activation='sigmoid')(dense)
    model = keras.Model(inputs=[q, qb, a], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=5e-3))
    # del embed
    # gc.collect()
    return model


In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_body)]
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, val_data, label):
        super(keras.callbacks.Callback, self).__init__()
        self.val_data = val_data
        self.score = 0
        self.maxscore = 0
        self.label = label
        self.patience = 3
        self.count = 0
    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        val_pred = self.model.predict(self.val_data, batch_size=16, verbose=1)
        score = compute_spearmanr(self.label, val_pred)
        print('Spearman - {:.5f}'.format(score))
        if self.maxscore>=0.3:
            self.patience=2
        if (score < self.score):
            self.score = score
            self.count+=1
            if self.count==self.patience:
                self.model.stop_training=True
        elif score > self.maxscore:
            self.score = score
            self.maxscore = score
            self.count = 0
            if self.maxscore > 0.3:
                self.model.save('drive/My Drive/GoogleQA/Models/{:}/use-model-{:}.h5'.format(save_model, i))
        else:
            self.score = score
            self.count = 0
          

In [0]:
with tf.Session() as session:
    keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    for i, (tr_idx, val_idx) in enumerate(idx, 1):
        print('\nFold - {:}\n'.format(i))
        tr, val = train.loc[tr_idx], train.loc[val_idx]
        tr_x = [tr['question_title'], tr['question_body'], tr['answer']]
        val_x = [val['question_title'], val['question_body'], val['answer']]
        ieval = IntervalEval(val_data=val_x, label=val[target_col].values)
        model = use_model()
        history = model.fit(
            tr_x,
            tr[target_col],
            epochs=100, 
            batch_size=16,
            callbacks=[ieval])
        del model, tr, val, tr_x, val_x
        gc.collect()

# with tf.Session() as session:
#     keras.backend.set_session(session)
#     session.run(tf.global_variables_initializer())  
#     session.run(tf.tables_initializer())
#     model.load_weights('use-model-1.h5')
#     preds = model.predict(val['sentence'], batch_size=16, verbose=1)
#     score=compute_spearmanr(val[target_col].values, preds)
#     print('Spearman - {:.5f}'.format(score))


Fold - 1

Train on 4863 samples
Epoch 1/100
1216/1216 [==============================] - 33s 27ms/sample
Spearman - 0.35849
4863/4863 [==============================] - 170s 35ms/sample - loss: 0.3946
Epoch 2/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.36453
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3720
Epoch 3/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37807
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3628
Epoch 4/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37832
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3549
Epoch 5/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37996
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3462
Epoch 6/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37266
4863/4863 [=======================